<a href="https://colab.research.google.com/github/nuno80/FoodDataCentral/blob/main/FoodDataCentral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests flask


In [12]:
import requests
import random

# Imposta la tua chiave API qui
API_KEY = 'lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp'  # Sostituisci con la tua chiave API

# Funzione per cercare cibi
def search_food(query):
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={API_KEY}&query={query}&dataType=Foundation,Survey%20(FNDDS),SR%20Legacy"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Funzione per ottenere i dettagli di più cibi
def get_food_details_bulk(fdc_ids):
    url = f"https://api.nal.usda.gov/fdc/v1/foods?api_key={API_KEY}"
    headers = {"Content-Type": "application/json"}
    data = {"fdcIds": fdc_ids}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Funzione per creare un piano pasti
def create_meal_plan(food_list, num_meals=3):
    meal_plan = []
    for _ in range(num_meals):
        meal = random.sample(food_list, 3)  # Seleziona 3 cibi a caso per ogni pasto
        meal_plan.append(meal)
    return meal_plan

# Funzione per estrarre nutrienti specifici
def extract_nutrients(food):
    nutrients = {}
    for nutrient in food.get('foodNutrients', []):
        nutrient_name = nutrient.get('nutrient', {}).get('name', None)
        if nutrient_name in ['Protein', 'Cholesterol', 'Fatty acids, total saturated']:
            nutrients[nutrient_name] = nutrient.get('amount', 'N/A')

    return {
        'description': food.get('description', 'N/A'),
        'proteins': nutrients.get('Protein', 'N/A'),
        'cholesterol': nutrients.get('Cholesterol', 'N/A'),
        'saturated_fats': nutrients.get('Fatty acids, total saturated', 'N/A')
    }


In [14]:
# Esegui una ricerca per un alimento specifico
query = 'Pizza'
food_data = search_food(query)

if food_data:
    food_list = food_data.get('foods', [])
    fdc_ids = [food.get('fdcId') for food in food_list]

    # Ottieni i dettagli nutrizionali per ogni cibo in blocco
    detailed_food_data = get_food_details_bulk(fdc_ids)

    detailed_food_list = []
    for food in detailed_food_data:
        detailed_food_list.append(extract_nutrients(food))

    # Stampa i primi 5 risultati della ricerca con dettagli nutrizionali
    print(f"Trovati {len(detailed_food_list)} cibi per '{query}':")
    for food in detailed_food_list[:5]:
        print(f"- {food['description']}: Proteine={food['proteins']}g, Colesterolo={food['cholesterol']}mg, Grassi Saturi={food['saturated_fats']}g")

    # Crea un piano pasti
    meal_plan = create_meal_plan(detailed_food_list)

    # Stampa il piano pasti generato
    print("\nPiano pasti generato:")
    for idx, meal in enumerate(meal_plan, start=1):
        print(f"\nPasto {idx}:")
        for food in meal:
            print(f"- {food['description']}: Proteine={food['proteins']}g, Colesterolo={food['cholesterol']}mg, Grassi Saturi={food['saturated_fats']}g")
else:
    print("Errore nella ricerca del cibo.")


Trovati 50 cibi per 'Pizza':
- Dessert pizza: Proteine=1.86g, Colesterolo=3.0mg, Grassi Saturi=1.92g
- Mexican pizza: Proteine=15.5g, Colesterolo=47.0mg, Grassi Saturi=4.54g
- Pizza rolls: Proteine=9.19g, Colesterolo=6.0mg, Grassi Saturi=2.62g
- PIZZA HUT 12" Cheese Pizza, Pan Crust: Proteine=11.73g, Colesterolo=21.0mg, Grassi Saturi=5.184g
- PIZZA HUT 12" Pepperoni Pizza, Pan Crust: Proteine=11.97g, Colesterolo=25.0mg, Grassi Saturi=5.271g

Piano pasti generato:

Pasto 1:
- PIZZA HUT 14" Pepperoni Pizza, Hand-Tossed Crust: Proteine=12.23g, Colesterolo=26.0mg, Grassi Saturi=5.404g
- Topping from meat pizza: Proteine=19.2g, Colesterolo=68.0mg, Grassi Saturi=10.6g
- Fast Food, Pizza Chain, 14" pizza, pepperoni topping, regular crust: Proteine=11.74g, Colesterolo=25.0mg, Grassi Saturi=5.092g

Pasto 2:
- School Lunch, pizza, TONY'S Breakfast Pizza Sausage, frozen: Proteine=10.39g, Colesterolo=20.0mg, Grassi Saturi=3.47g
- Topping from meat and vegetable pizza: Proteine=17.6g, Colesterolo=6